In [31]:
import datetime as dt
import ast
import json
from collections import Counter
import os

import numpy as np
import pandas as pd
import string
import re

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import tensorflow as tf
from tensorflow import keras
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential, Model
from keras.layers import Input, Embedding, Flatten, Dense, concatenate
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.cluster import KMeans
from sklearn import preprocessing
import statsmodels.api as sm

In [74]:
df = pd.read_csv('new_cars.csv')
#print(df.dtypes)

In [3]:
### Drop unncessary columns
columns_to_drop = ['link', 'stock_number', 'vin', 'deal_gauge', 'exterior_color', 'mpg', 'fuel_type', 'engine', 'price_history', 'vehicle_history', 'listing_title', 'interior_color']
df = df.drop(columns=columns_to_drop)

columns_to_drop = ['Open recall', 'Accidents or damage']
df = df.drop(columns=columns_to_drop)

In [4]:
### Convert data types
boolean_columns = ['1-owner vehicle', 'Personal use only', 'Clean title']

for column in boolean_columns:
    df[column] = df[column].map({'Yes': True, 'No': False}).astype(bool)

In [5]:
# Check for NAs
missing_counts = df.isna().sum()
print(missing_counts[missing_counts > 0])

seller_name                  44
listed_date                1522
price_change               1522
price_change_percentage    1522
Cylinders                    73
Liters                      105
exterior_color_1            227
exterior_color_2           5989
interior_color_1            539
interior_color_2           5796
low mpg                     424
high mpg                    424
dtype: int64


In [6]:
# Drop rows with a low number of missing values
columns_with_missing_values = ['seller_name', 'Cylinders', 'Liters', 'exterior_color_1', 'low mpg', 'high mpg', 'interior_color_1']
df = df.dropna(subset=columns_with_missing_values)

# Drop exterior_color_2 since it only has 12 values
df = df.drop(columns=['exterior_color_2'])
# Drop interior_color_2 since it only has 203 values
df = df.drop(columns=['interior_color_2'])

# Drop listed_date, price_change, price_change_percentage because they account for 25% of data, 
# and they are not correlated with price in the correlation matrix.
columns_to_drop = ['listed_date', 'price_change', 'price_change_percentage']
df = df.drop(columns=columns_to_drop)

In [7]:
### Code related to price_history, which we decided to remove

# Convert listed_date to Days Avaliable
# current_date = pd.Timestamp(dt.datetime.now().date())
# df['listed_date'] = pd.to_datetime(df['listed_date'])
# df['Days Avaliable'] = (current_date - df['listed_date']).dt.days
# df = df.drop('listed_date', axis=1)
# df.head()

# Drop missing price history data. ~25% of data.
# columns_with_missing_values = ['price_change', 'price_change_percentage']
# df = df.dropna(subset=columns_with_missing_values)

# Drop 'price_change' and 'price_change_percentage' to avoid data leakage
# columns_to_drop = ['price_change', 'price_change_percentage']
# df = df.drop(columns=columns_to_drop)

In [8]:
df.head()

,listing_mileage,primary_price,drivetrain,transmission,seller_name,Accidents or damage,1-owner vehicle,Personal use only,Open recall,Clean title,year,make,model,Cylinders,Liters,Boosted,exterior_color_1,interior_color_1,low mpg,high mpg
0,79497,17899,Front-wheel Drive,6-Speed Automatic,ETHIO Motors,False,True,False,True,True,2018,Hyundai,Santa Fe Sport 2.4L,4.0,2.4,False,blue,beige,21.0,27.0
1,80326,27900,Four-wheel Drive,8-Speed Automatic,Custom Cars West,False,False,True,False,True,2017,RAM,1500 Express,8.0,5.7,False,silver,black,16.0,23.0
2,92959,18950,Front-wheel Drive,Automatic CVT,Custom Cars West,False,False,True,False,True,2019,Nissan,Kicks SR,4.0,1.6,False,black,gray,31.0,36.0
3,59176,39950,Four-wheel Drive,8-Speed Automatic,Custom Cars West,True,False,False,True,True,2019,RAM,1500 Laramie,8.0,5.7,False,gray,black,19.0,24.0
4,135385,26950,Four-wheel Drive,8-Speed Automatic,Custom Cars West,False,False,True,True,True,2015,Jeep,Wrangler Unlimited Rubicon,6.0,3.6,False,blue,black,16.0,21.0


In [9]:
df.shape

(4801, 20)

### outcome variable: primary_price
### ocluded variables: price_change	price_change_percentage

## Categorical variables

In [10]:
categorical_columns = ['seller_name', 'drivetrain', 'transmission', 'make', 'model', 'exterior_color_1', 'interior_color_1']

# Check how many unique values there are in categorical_columns
for column in categorical_columns:
    unique_values = df[column].nunique()
    print(f"Unique values for {column}:", unique_values)

# Models has a large number of unique values
# seller_name, transmission, make, model have a large number of unique values
# Consider embedding these instead of one-hot encode

Unique values for seller_name: 122
Unique values for drivetrain: 5
Unique values for transmission: 105
Unique values for make: 49
Unique values for model: 1620
Unique values for exterior_color_1: 13
Unique values for interior_color_1: 11


In [11]:
X = df.drop('primary_price', axis=1)  # Features
y = df['primary_price']  # Target

In [12]:
print(X.dtypes)
print(y.dtype)

listing_mileage          int64
drivetrain              object
transmission            object
seller_name             object
Accidents or damage       bool
1-owner vehicle           bool
Personal use only         bool
Open recall               bool
Clean title               bool
year                     int64
make                    object
model                   object
Cylinders              float64
Liters                 float64
Boosted                   bool
exterior_color_1        object
interior_color_1        object
low mpg                float64
high mpg               float64
dtype: object
int64


In [13]:
### One-hot Encode 
X = pd.get_dummies(X, columns=categorical_columns)

In [14]:
X.shape

(4801, 1937)

In [15]:
# Check that X doesn't have object data
unsupported_types = []

for column in X.columns:
    dtype = X[column].dtype
    if dtype == 'object':
        unsupported_types.append((column, dtype))

print("Unsupported Data Types:")
for column, dtype in unsupported_types:
    print(f"{column}: {dtype}")


Unsupported Data Types:


In [16]:
df.head()

,listing_mileage,primary_price,drivetrain,transmission,seller_name,Accidents or damage,1-owner vehicle,Personal use only,Open recall,Clean title,year,make,model,Cylinders,Liters,Boosted,exterior_color_1,interior_color_1,low mpg,high mpg
0,79497,17899,Front-wheel Drive,6-Speed Automatic,ETHIO Motors,False,True,False,True,True,2018,Hyundai,Santa Fe Sport 2.4L,4.0,2.4,False,blue,beige,21.0,27.0
1,80326,27900,Four-wheel Drive,8-Speed Automatic,Custom Cars West,False,False,True,False,True,2017,RAM,1500 Express,8.0,5.7,False,silver,black,16.0,23.0
2,92959,18950,Front-wheel Drive,Automatic CVT,Custom Cars West,False,False,True,False,True,2019,Nissan,Kicks SR,4.0,1.6,False,black,gray,31.0,36.0
3,59176,39950,Four-wheel Drive,8-Speed Automatic,Custom Cars West,True,False,False,True,True,2019,RAM,1500 Laramie,8.0,5.7,False,gray,black,19.0,24.0
4,135385,26950,Four-wheel Drive,8-Speed Automatic,Custom Cars West,False,False,True,True,True,2015,Jeep,Wrangler Unlimited Rubicon,6.0,3.6,False,blue,black,16.0,21.0


## Split data into 80% training set and 20% validation set.

In [17]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert all columns to float
X_train = X_train.astype(float)
X_val = X_val.astype(float)
y_train = y_train.astype(float)
y_val = y_val.astype(float)

# Convert data to numpy array
X_train = X_train.to_numpy()
X_val = X_val.to_numpy()
y_train = y_train.to_numpy()
y_val = y_val.to_numpy()

In [18]:
print("X_train shape:", X_train.shape)
print("X_val shape:", X_val.shape)
print("y_train shape:", y_train.shape)
print("y_val shape:", y_val.shape)

X_train shape: (3840, 1937)
X_val shape: (961, 1937)
y_train shape: (3840,)
y_val shape: (961,)


In [19]:
# Convert NumPy arrays to PyTorch tensors
X_train_tensor = torch.Tensor(X_train)
y_train_tensor = torch.Tensor(y_train)
X_val_tensor = torch.Tensor(X_val)
y_val_tensor = torch.Tensor(y_val)

## Deep Neural Net

In [20]:
### --- First attempt ---

# class NeuralNetwork(nn.Module):
#     def __init__(self, input_size):
#         super(NeuralNetwork, self).__init__()
#         self.fc1 = nn.Linear(input_size, 256)
#         self.relu1 = nn.ReLU()
#         self.dropout1 = nn.Dropout(0.5)
#         self.fc2 = nn.Linear(256, 128)
#         self.relu2 = nn.ReLU()
#         self.dropout2 = nn.Dropout(0.5)
#         self.fc3 = nn.Linear(128, 64)
#         self.relu3 = nn.ReLU()
#         self.dropout3 = nn.Dropout(0.5)
#         self.fc4 = nn.Linear(64, 1)  # Output layer for regression

#     def forward(self, x):
#         x = self.fc1(x)
#         x = self.relu1(x)
#         x = self.dropout1(x)
#         x = self.fc2(x)
#         x = self.relu2(x)
#         x = self.dropout2(x)
#         x = self.fc3(x)
#         x = self.relu3(x)
#         x = self.dropout3(x)
#         x = self.fc4(x)
#         return x

# # Create the model
# input_size = X_train_tensor.shape[1]
# model = NeuralNetwork(input_size)

# # Loss function for regression
# criterion = nn.MSELoss()

# # Optimizer
# optimizer = optim.Adam(model.parameters(), lr=0.001)

In [51]:
class NeuralNetwork(nn.Module):
    def __init__(self, hidden_layers, neurons_per_layer, dropout_rate):
        super(NeuralNetwork, self).__init__()
        self.fc_layers = nn.ModuleList()
        input_size = X_train_tensor.shape[1]

        # Add hidden layers
        for _ in range(hidden_layers):
            self.fc_layers.append(nn.Linear(input_size, neurons_per_layer))
            self.fc_layers.append(nn.ReLU())
            self.fc_layers.append(nn.Dropout(dropout_rate))
            input_size = neurons_per_layer

        # Output layer
        self.fc_layers.append(nn.Linear(input_size, 1))

    def forward(self, x):
        for layer in self.fc_layers:
            x = layer(x)
        return x

def create_model(hidden_layers, neurons_per_layer, dropout_rate):
    return NeuralNetwork(hidden_layers, neurons_per_layer, dropout_rate)

In [54]:
# Training loop function
def train_DNN(epochs, benchmark, model, optimizer):
    #training_losses = []
    criterion = nn.MSELoss()

    for epoch in range(epochs):
        # Forward pass
        outputs = model(X_train_tensor)
        loss = criterion(outputs, y_train_tensor.view(-1, 1))
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Calculate training accuracy
        with torch.no_grad():
            train_outputs = model(X_train_tensor)
            train_loss = criterion(train_outputs, y_train_tensor.view(-1, 1))
        
        # Record training loss and MAE
        #if (epoch+1) % benchmark == 0:
            #print(f'Epoch [{epoch+1}/{epochs}], Training Loss: {train_loss.item()}')
            #mae = torch.abs(outputs - y_train_tensor.view(-1, 1)).mean().item()
            #print(f'MAE: {int(mae)}')

            #training_losses.append((epoch + 1, train_loss.item(), mae))

    #return training_losses

In [57]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def evaluate_model(model, X, y):
    # Forward pass
    outputs = model(X)
    
    # Calculate loss
    criterion = nn.MSELoss()
    loss = criterion(outputs, y.view(-1, 1))
    
    # Convert predictions and targets to numpy arrays
    y_pred = outputs.detach().numpy()
    y_true = y.numpy()
    
    # Calculate metrics
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    
    return loss.item(), mae, mse, rmse, r2

# After the training loop

# Evaluate the model on the training set
# train_loss, train_mae, train_mse, train_rmse, train_r2 = evaluate_model(model, criterion, X_train_tensor, y_train_tensor)

# # Evaluate the model on the validation set
# val_loss, val_mae, val_mse, val_rmse, val_r2 = evaluate_model(model, criterion, X_val_tensor, y_val_tensor)

# # Print the evaluation metrics
# print(f"Training:\nLoss: {train_loss:.4f}\nMAE: {train_mae:.4f}\nRMSE: {train_rmse:.4f}\nR^2: {train_r2:.4f}\n")
# print(f"Validation:\nLoss: {val_loss:.4f}\nMAE: {val_mae:.4f}\nRMSE: {val_rmse:.4f}\nR^2: {val_r2:.4f}")

In [63]:
### Iterate through parameters

# 3^4=81
learning_rates = [0.01, 0.001, 0.0001]
learning_rates = [0.001, 0.0001]
hidden_layers_list = [1, 2, 3]
neurons_per_layer_list = [64, 128, 256]
dropout_rates = [0.1, 0.3, 0.5]

try:
    with open('results.json', 'r') as file:
        results = json.load(file)
except FileNotFoundError:
    results = []

for lr in learning_rates:
    for hidden_layers in hidden_layers_list:
        for neurons_per_layer in neurons_per_layer_list:
            for dropout_rate in dropout_rates:
                # Build and train the model with the current hyperparameters
                #print("--------------------------------------------------------------------------------------------------------")
                print(f"lr={lr}, hidden_layers={hidden_layers}, neurons_per_layer={neurons_per_layer}, dropout_rate={dropout_rate}")
                #model_save_path = f'models/{hidden_layers}_{neurons_per_layer}_{dropout_rate}'
                model = create_model(hidden_layers, neurons_per_layer, dropout_rate)
                optimizer = optim.Adam(model.parameters(), lr=lr)
                
                # Training loop 
                train_DNN(1000, 50, model, optimizer)
                #plot_loss_mae(training_loss_mae, model_save_path)

                # Evaluate the model on the training set
                train_loss, train_mae, train_mse, train_rmse, train_r2 = evaluate_model(model, X_train_tensor, y_train_tensor)
                #print(f"Training:\nLoss: {train_loss:.4f}\nMAE: {train_mae:.4f}\nRMSE: {train_rmse:.4f}\nR^2: {train_r2:.4f}\n")

                # Evaluate the model on the validation set
                val_loss, val_mae, val_mse, val_rmse, val_r2 = evaluate_model(model, X_val_tensor, y_val_tensor)
                #print(f"Validation:\nLoss: {val_loss:.4f}\nMAE: {val_mae:.4f}\nRMSE: {val_rmse:.4f}\nR^2: {val_r2:.4f}")

                result_entry = {
                    'lr': lr,
                    'hidden_layers': hidden_layers,
                    'neurons_per_layer': neurons_per_layer,
                    'dropout_rate': dropout_rate,
                    'train_loss': train_loss,
                    'train_mae': train_mae,
                    'train_mse': train_mse,
                    'train_rmse': train_rmse,
                    'train_r2': train_r2,
                    'val_loss': val_loss,
                    'val_mae': val_mae,
                    'val_mse': val_mse,
                    'val_rmse': val_rmse,
                    'val_r2': val_r2,
                    #'training_losses': training_losses
                }
                
                results.append(result_entry)

with open('results.json', 'w') as file:
    json.dump(results, file)

lr=0.01, hidden_layers=3, neurons_per_layer=128, dropout_rate=0.1
lr=0.01, hidden_layers=3, neurons_per_layer=128, dropout_rate=0.3
lr=0.01, hidden_layers=3, neurons_per_layer=128, dropout_rate=0.5
lr=0.01, hidden_layers=3, neurons_per_layer=256, dropout_rate=0.1
lr=0.01, hidden_layers=3, neurons_per_layer=256, dropout_rate=0.3
lr=0.01, hidden_layers=3, neurons_per_layer=256, dropout_rate=0.5
lr=0.001, hidden_layers=3, neurons_per_layer=128, dropout_rate=0.1


KeyboardInterrupt: 

In [70]:
### Check best performing model:
try:
    with open('results.json', 'r') as file:
        results = json.load(file)
except FileNotFoundError:
    print("No results.json")

best_train_loss = float('inf')
best_val_loss = float('inf')
best_train_mae = float('inf')
best_val_mae = float('inf')

for result in results:
    if result['train_loss'] < best_train_loss:
        best_train_loss = result['train_loss']
        best_train_loss_result = result
    
    if result['val_loss'] < best_val_loss:
        best_val_loss = result['val_loss']
        best_val_loss_result = result

    if result['train_mae'] < best_train_mae:
        best_train_mae = result['train_mae']
        best_train_mae_result = result
    
    if result['val_mae'] < best_val_mae:
        best_val_mae = result['val_mae']
        best_val_mae_result = result

# Print the best result for each metric
print("Best Train Loss Result:")
print(best_train_loss_result)

print("\nBest Validation Loss Result:")
print(best_val_loss_result)

print("\nBest Train MAE Result:")
print(best_train_mae_result)

print("\nBest Validation MAE Result:")
print(best_val_mae_result)

Best Train Loss Result:
{'lr': 0.01, 'hidden_layers': 2, 'neurons_per_layer': 256, 'dropout_rate': 0.1, 'train_loss': 195805296.0, 'train_mae': 8089.2456, 'train_mse': 195805300.0, 'train_rmse': 13993.045, 'train_r2': 0.6620980992814658, 'val_loss': 316307776.0, 'val_mae': 8815.483, 'val_mse': 316307780.0, 'val_rmse': 17785.043, 'val_r2': 0.5439361241808669}

Best Validation Loss Result:
{'lr': 0.01, 'hidden_layers': 2, 'neurons_per_layer': 256, 'dropout_rate': 0.1, 'train_loss': 195805296.0, 'train_mae': 8089.2456, 'train_mse': 195805300.0, 'train_rmse': 13993.045, 'train_r2': 0.6620980992814658, 'val_loss': 316307776.0, 'val_mae': 8815.483, 'val_mse': 316307780.0, 'val_rmse': 17785.043, 'val_r2': 0.5439361241808669}

Best Train MAE Result:
{'lr': 0.01, 'hidden_layers': 1, 'neurons_per_layer': 256, 'dropout_rate': 0.1, 'train_loss': 285936096.0, 'train_mae': 6984.467, 'train_mse': 285936130.0, 'train_rmse': 16909.646, 'train_r2': 0.5065590364401802, 'val_loss': 375028512.0, 'val_mae':

In [22]:
# Function to save plots for training loss and MAE
def plot_loss_mae(training_loss_mae, save_path):
    epochs, losses, mae = zip(*training_loss_mae)

    # Save the Training Loss plot
    plt.plot(epochs, losses, label='Training Loss')
    plt.title('Training Loss Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Training Loss')
    plt.legend()
    plt.savefig(save_path + '__loss.png')
    plt.close()

    # Save the MAE plot
    plt.plot(epochs, mae, label='MAE')
    plt.title('MAE Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('MAE')
    plt.legend()
    plt.savefig(save_path + '__mae.png')
    plt.close()

In [24]:
# %%time
# epochs = 1000                           # How many times to run the training loop
# benchmark = 50                          # Breakpoint for output
# model_save_path = 'DNN_model.pth'       # Where the model is saved

# training_losses = train_DNN(epochs, benchmark, model_save_path)

In [22]:
# Load the model (so it doesn't have to retrain every time)
model_save_path = 'DNN_model.pth'
model.load_state_dict(torch.load(model_save_path))

<All keys matched successfully>

## Linear Regression

In [71]:
# Setup model
m = LinearRegression()
# Give input and output data
m.fit(X_train, y_train)
print('Training accuracy (the coefficient of determination): {:.3f}'.format(m.score(X_train, y_train)))

Training accuracy (the coefficient of determination): 0.983


#### Test the model on the test data

In [72]:
prediction = m.predict(X_val)

In [73]:
print('Testing accuracy by using score function: {:.3f}'.format(m.score(X_val, y_val)))
print('Testing accuracy by using r2_score meric: {:.3f}'.format(r2_score(y_val, m.predict(X_val))))

Testing accuracy by using score function: -211569106.373
Testing accuracy by using r2_score meric: -211569106.373


In [22]:
mean_absolute_error(y_val, prediction)

90906898.76073161

#### Add statsmodel for p-value

In [23]:
X_train_with_constant = sm.add_constant(X_train)
model = sm.OLS(y_train, X_train_with_constant).fit()
# Get the summary of the regression

#### Change the variables to the actual variable name

In [24]:
xnames = ['const']
xnames.extend(list(X.columns))

In [25]:
exog_names = model.model.exog_names

In [26]:
dic = dict(zip(exog_names, xnames))

In [27]:
def rename_vars(vname): 
    to_ret = vname
    for orig_vname in list(dic.keys()):
        if vname == orig_vname:
            to_ret = dic[orig_vname] 
    return to_ret

def rename_ols(reg): 
    for i in range(len(reg)): 
        reg[i] = rename_vars(reg[i])

In [28]:
rename_ols(model.model.exog_names)

In [29]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.983
Model:                            OLS   Adj. R-squared:                  0.970
Method:                 Least Squares   F-statistic:                     74.37
Date:                Thu, 16 Nov 2023   Prob (F-statistic):               0.00
Time:                        16:37:47   Log-Likelihood:                -36385.
No. Observations:                3840   AIC:                         7.611e+04
Df Residuals:                    2169   BIC:                         8.656e+04
Df Model:                        1670                                         
Covariance Type:            nonrobust                                         
                                                                                    coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------

## Predict deal gauge using K-means

In [2]:
df = pd.read_csv('new_cars.csv')
df.head()

,link,listing_title,listing_mileage,primary_price,deal_gauge,exterior_color,interior_color,drivetrain,mpg,fuel_type,...,price_change_percentage,Cylinders,Liters,Boosted,exterior_color_1,exterior_color_2,interior_color_1,interior_color_2,low mpg,high mpg
0,"""https://www.cars.com/vehicledetail/5ee3ae04-6...",2018 Hyundai Santa Fe Sport 2.4L,79497,17899,Good Deal $2 under,Nightfall Blue,Beige,Front-wheel Drive,21-27,Gasoline,...,-11.823243,4.0,2.4,False,blue,NaN,beige,NaN,21.0,27.0
1,"""https://www.cars.com/vehicledetail/c99e5035-7...",2017 RAM 1500 Express,80326,27900,This is a good deal.,Silver,Black,Four-wheel Drive,16-23,Gasoline,...,-9.956431,8.0,5.7,False,silver,NaN,black,NaN,16.0,23.0
2,"""https://www.cars.com/vehicledetail/4e79fa3a-b...",2019 Nissan Kicks SR,92959,18950,This is a fair deal. Why?,Monarch Orange / Super Black,Charcoal,Front-wheel Drive,31-36,Gasoline,...,-5.155155,4.0,1.6,False,black,orange,gray,NaN,31.0,36.0
3,"""https://www.cars.com/vehicledetail/86a0d096-3...",2019 RAM 1500 Laramie,59176,39950,Great Deal $968 under,Gray,Black,Four-wheel Drive,19-24,Gasoline,...,-1.414012,8.0,5.7,False,gray,NaN,black,NaN,19.0,24.0
4,"""https://www.cars.com/vehicledetail/36efbef5-f...",2015 Jeep Wrangler Unlimited Rubicon,135385,26950,This is a good deal.,Blue,Black,Four-wheel Drive,16-21,Gasoline,...,-10.016694,6.0,3.6,False,blue,NaN,black,NaN,16.0,21.0


In [3]:
# converting deal gauge to Deal
def get_deal(val):
    val = str(val).lower()
    s_list = val.split(" ")
    if "good" in s_list:
        return "Good"
    elif "great" in s_list:
        return "Great"
    elif "fair" in s_list:
        return "Fair"
    else:
        return "No Assessment Given"

df['Deal'] = df.deal_gauge.apply(lambda x: get_deal(x))
df.head()

,link,listing_title,listing_mileage,primary_price,deal_gauge,exterior_color,interior_color,drivetrain,mpg,fuel_type,...,Cylinders,Liters,Boosted,exterior_color_1,exterior_color_2,interior_color_1,interior_color_2,low mpg,high mpg,Deal
0,"""https://www.cars.com/vehicledetail/5ee3ae04-6...",2018 Hyundai Santa Fe Sport 2.4L,79497,17899,Good Deal $2 under,Nightfall Blue,Beige,Front-wheel Drive,21-27,Gasoline,...,4.0,2.4,False,blue,NaN,beige,NaN,21.0,27.0,Good
1,"""https://www.cars.com/vehicledetail/c99e5035-7...",2017 RAM 1500 Express,80326,27900,This is a good deal.,Silver,Black,Four-wheel Drive,16-23,Gasoline,...,8.0,5.7,False,silver,NaN,black,NaN,16.0,23.0,Good
2,"""https://www.cars.com/vehicledetail/4e79fa3a-b...",2019 Nissan Kicks SR,92959,18950,This is a fair deal. Why?,Monarch Orange / Super Black,Charcoal,Front-wheel Drive,31-36,Gasoline,...,4.0,1.6,False,black,orange,gray,NaN,31.0,36.0,Fair
3,"""https://www.cars.com/vehicledetail/86a0d096-3...",2019 RAM 1500 Laramie,59176,39950,Great Deal $968 under,Gray,Black,Four-wheel Drive,19-24,Gasoline,...,8.0,5.7,False,gray,NaN,black,NaN,19.0,24.0,Great
4,"""https://www.cars.com/vehicledetail/36efbef5-f...",2015 Jeep Wrangler Unlimited Rubicon,135385,26950,This is a good deal.,Blue,Black,Four-wheel Drive,16-21,Gasoline,...,6.0,3.6,False,blue,NaN,black,NaN,16.0,21.0,Good


In [4]:
# Drop unncessary columns
columns_to_drop = ['link', 'stock_number', 'vin', 'deal_gauge', 'exterior_color', 'mpg', 'fuel_type', 'engine', 'price_history', 'vehicle_history', 'listing_title', 'interior_color']
df = df.drop(columns=columns_to_drop)

In [5]:
# Convert data types
boolean_columns = ['1-owner vehicle', 'Personal use only', 'Clean title']

for column in boolean_columns:
    df[column] = df[column].map({'Yes': True, 'No': False}).astype(bool)

In [6]:
# Check for NAs
missing_counts = df.isna().sum()
print(missing_counts[missing_counts > 0])

seller_name                  44
listed_date                1522
price_change               1522
price_change_percentage    1522
Cylinders                    73
Liters                      105
exterior_color_1            227
exterior_color_2           5989
interior_color_1            539
interior_color_2           5796
low mpg                     424
high mpg                    424
dtype: int64


In [7]:
# Drop rows with a low number of missing values
columns_with_missing_values = ['seller_name', 'Cylinders', 'Liters', 'exterior_color_1', 'low mpg', 'high mpg', 'interior_color_1']
df = df.dropna(subset=columns_with_missing_values)

# Drop exterior_color_2 since it only has 12 values
df = df.drop(columns=['exterior_color_2'])
# Drop interior_color_2 since it only has 203 values
df = df.drop(columns=['interior_color_2'])

# Drop listed_date, price_change, price_change_percentage because they account for 25% of data, 
# and they are not correlated with price in the correlation matrix.
columns_to_drop = ['listed_date', 'price_change', 'price_change_percentage']
df = df.drop(columns=columns_to_drop)

In [8]:
df.head()

,listing_mileage,primary_price,drivetrain,transmission,seller_name,Accidents or damage,1-owner vehicle,Personal use only,Open recall,Clean title,...,make,model,Cylinders,Liters,Boosted,exterior_color_1,interior_color_1,low mpg,high mpg,Deal
0,79497,17899,Front-wheel Drive,6-Speed Automatic,ETHIO Motors,False,True,False,True,True,...,Hyundai,Santa Fe Sport 2.4L,4.0,2.4,False,blue,beige,21.0,27.0,Good
1,80326,27900,Four-wheel Drive,8-Speed Automatic,Custom Cars West,False,False,True,False,True,...,RAM,1500 Express,8.0,5.7,False,silver,black,16.0,23.0,Good
2,92959,18950,Front-wheel Drive,Automatic CVT,Custom Cars West,False,False,True,False,True,...,Nissan,Kicks SR,4.0,1.6,False,black,gray,31.0,36.0,Fair
3,59176,39950,Four-wheel Drive,8-Speed Automatic,Custom Cars West,True,False,False,True,True,...,RAM,1500 Laramie,8.0,5.7,False,gray,black,19.0,24.0,Great
4,135385,26950,Four-wheel Drive,8-Speed Automatic,Custom Cars West,False,False,True,True,True,...,Jeep,Wrangler Unlimited Rubicon,6.0,3.6,False,blue,black,16.0,21.0,Good


In [9]:
df_map = {}  
cols = df.columns.values
print('cols:',cols)
for col in cols:
    if df[col].dtype != np.int64 and df[col].dtype != np.float64:
        temp = {}
        x = 0
        for ele in set(df[col].values.tolist()):
            if ele not in temp:
                temp[ele] = x
                x += 1

        df_map[df[col].name] = temp
        df[col] = list(map(lambda val: temp[val], df[col]))

for key, value in df_map.items():
   print(key,value)

cols: ['listing_mileage' 'primary_price' 'drivetrain' 'transmission'
 'seller_name' 'Accidents or damage' '1-owner vehicle' 'Personal use only'
 'Open recall' 'Clean title' 'year' 'make' 'model' 'Cylinders' 'Liters'
 'Boosted' 'exterior_color_1' 'interior_color_1' 'low mpg' 'high mpg'
 'Deal']
drivetrain {'All-wheel Drive': 0, 'Four-wheel Drive': 1, 'Rear-wheel Drive': 2, 'Front-wheel Drive': 3, 'Unknown': 4}
transmission {'7-Speed': 0, 'Auto, 8-Spd CmdShft': 1, 'Auto, 6-Spd EcoShift': 2, 'Auto, 8-Spd Spt Stptrnc': 3, '7-Speed Automatic': 4, 'Automatic, CVT w/Xtronic': 5, 'Automatic with Geartronic': 6, 'Automatic, 10-Spd': 7, '7-Speed Porsche Doppelkupplung (PDK)': 8, 'Auto, 7-Spd M DCT w/Drivelogic': 9, 'Manual': 10, 'Automatic, CVT': 11, 'CVT 7-Speed': 12, '7-Speed Shiftable Automatic': 13, 'Automatic': 14, 'Automatic, 6-Spd w/Overdrive': 15, 'Manual, 6-Spd': 16, 'Auto, 6-Spd w/Overdrive': 17, '8-Speed Automatic Sport': 18, '8-Speed Automatic with Auto-Shift': 19, '8-Speed Shiftable

In [10]:
df.head()

,listing_mileage,primary_price,drivetrain,transmission,seller_name,Accidents or damage,1-owner vehicle,Personal use only,Open recall,Clean title,...,make,model,Cylinders,Liters,Boosted,exterior_color_1,interior_color_1,low mpg,high mpg,Deal
0,79497,17899,3,50,69,0,1,0,1,1,...,0,530,4.0,2.4,0,7,2,21.0,27.0,1
1,80326,27900,1,103,111,0,0,1,0,1,...,24,346,8.0,5.7,0,2,3,16.0,23.0,1
2,92959,18950,3,91,111,0,0,1,0,1,...,11,1191,4.0,1.6,0,3,5,31.0,36.0,2
3,59176,39950,1,103,111,1,0,0,1,1,...,24,720,8.0,5.7,0,5,3,19.0,24.0,3
4,135385,26950,1,103,111,0,0,1,1,1,...,19,314,6.0,3.6,0,7,3,16.0,21.0,1


In [11]:
# apply the k-means model
x = np.array(df.drop(['Deal'], 1).astype(float))
x = preprocessing.scale(x)

clf = KMeans(n_clusters=3)
clf.fit(x)

y = np.array(df['Deal'])

/var/folders/cd/7yklxg1d61j6p5hx348c94tw0000gn/T/ipykernel_43507/1314405606.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  x = np.array(df.drop(['Deal'], 1).astype(float))
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [12]:
# calulate accuracy
correct = 0
for i in range(len(x)):
    predict_data = np.array(x[i].astype(float))
    predict_data = predict_data.reshape(-1, len(predict_data))
    predict = clf.predict(predict_data)
    # print(predict[0], y[i])
    if predict[0] == y[i]:
        correct += 1

print(f"accuracy :{correct * 1.0 / len(x)}")

accuracy :0.320766506977713
